In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
a = torch.tensor(torch.range(1,9),requires_grad=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [3]:
1/a.t()

tensor([1.0000, 0.5000, 0.3333, 0.2500, 0.2000, 0.1667, 0.1429, 0.1250, 0.1111],
       grad_fn=<MulBackward0>)

In [14]:
a_sp =torch.sparse_coo_tensor(a.view((3,3)).nonzero().t(),a,torch.Size([3,3]))
print(a_sp)
print(a_sp._indices())
print(a_sp._values())
print(a_sp.size())

tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 1, 2, 0, 1, 2, 0, 1, 2]])
tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.])
torch.Size([3, 3])


In [5]:
b = torch.sparse.sum(a_sp,0).to_dense()
print(b)

tensor([12., 15., 18.], grad_fn=<ToDenseBackward>)


In [6]:
b.backward(gradient=torch.tensor([2.,1.,3.]))

In [7]:
print(b.grad)
print(a_sp.grad)
print(a.grad)

None
None
tensor([2., 1., 3., 2., 1., 3., 2., 1., 3.])


In [8]:
h = torch.ones((1,9),requires_grad=True).view(3,3)
h_p = a_sp.mm(h).div(b)
print(h_p)

tensor([[0.5000, 0.4000, 0.3333],
        [1.2500, 1.0000, 0.8333],
        [2.0000, 1.6000, 1.3333]], grad_fn=<DivBackward0>)


In [9]:
# class SpGraphAttentionLayer(nn.Module):
#     """
#     Sparse version GAT layer, similar to https://arxiv.org/abs/1710.10903
#     """

#     def __init__(self, in_features, out_features, dropout, alpha):
#         super(SpGraphAttentionLayer, self).__init__()
#         self.in_features = in_features
#         self.out_features = out_features
#         self.alpha = alpha

#         self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
#         nn.init.xavier_normal_(self.W.data, gain=1.414)
                
#         self.a = nn.Parameter(torch.zeros(size=(1, 2*out_features)))
#         nn.init.xavier_normal_(self.a.data, gain=1.414)

#         self.dropout = nn.Dropout(dropout)
#         self.leakyrelu = nn.LeakyReLU(self.alpha)

#     def forward(self, input, edge):
#         dv = 'cuda' if input.is_cuda else 'cpu'

#         N = input.size()[0]

#         h = torch.mm(input, self.W)
#         # h: N x out
#         assert not torch.isnan(h).any()

#         # Self-attention on the nodes - Shared attention mechanism
#         edge_h = torch.cat((h[edge[0,:], :], h[edge[1,:], :]), dim=1).t()
#         # edge_h: 2*D x E

#         edge_e = torch.exp(-self.leakyrelu(self.a.mm(edge_h).squeeze()))
#         assert not torch.isnan(edge_e).any()
#         # edge_e: E

#         edge_e = self.dropout(edge_e)

#         edge_e_sparse = torch.sparse_coo_tensor(edge, edge_e, [N, N])
#         e_rowsum = torch.sparse.sum(edge_e_sparse,1).to_dense().view((-1,1))
#         # e_rowsum: N x 1
        
#         print(e_rowsum)
        
#         h_p = edge_e_sparse.mm(h)
#         assert not torch.isnan(h_p).any()
#         # h_prime: N x out
        
#         print(h_p)
        
#         h_prime = h_p.div(e_rowsum)
#         # h_prime: N x out
#         print(h_prime)
#         h_prime[h_p==0] = 0
#         assert not torch.isnan(h_prime).any()

#         return F.elu(h_prime)

#     def __repr__(self):
#         return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [10]:


class SpecialSpmmFunction(torch.autograd.Function):
    """Special function for only sparse region backpropataion layer."""
    @staticmethod
    def forward(ctx, indices, values, shape, b):
        assert indices.requires_grad == False
        a = torch.sparse_coo_tensor(indices, values, shape)
        a_r = torch.sparse_coo_tensor(indices, [1/v if v != 0 else 0 for v in values], shape)
        ctx.save_for_backward(a_r, 1/b)
        ctx.N = shape[0]
        return torch.matmul(a, b)

    @staticmethod
    def backward(ctx, grad_output):
        a, b = ctx.saved_tensors
        grad_values = grad_b = None
        if ctx.needs_input_grad[1]:
            grad_a_dense = grad_output.mm(1/b.t())
            edge_idx = a._indices()[0, :] * ctx.N + a._indices()[1, :]
            grad_values = grad_a_dense.view(-1)[edge_idx]
        if ctx.needs_input_grad[3]:
            grad_b = (1/a.t()).mm(grad_output)
        return None, grad_values, None, grad_b


class SpecialSpmm(nn.Module):
    def forward(self, indices, values, shape, b):
        return SpecialSpmmFunction.apply(indices, values, shape, b)


class SpGraphAttentionLayer(nn.Module):
    """
    Sparse version GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(SpGraphAttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_normal_(self.W.data, gain=1.414)
                
        self.a = nn.Parameter(torch.zeros(size=(1, 2*out_features)))
        nn.init.xavier_normal_(self.a.data, gain=1.414)

        self.dropout = nn.Dropout(dropout)
        self.leakyrelu = nn.LeakyReLU(self.alpha)
        self.special_spmm = SpecialSpmm()

    def forward(self, input, edge):
        dv = 'cuda' if input.is_cuda else 'cpu'

        N = input.size()[0]

        h = torch.mm(input, self.W)
        # h: N x out
        assert not torch.isnan(h).any()

        # Self-attention on the nodes - Shared attention mechanism
        edge_h = torch.cat((h[edge[0, :], :], h[edge[1, :], :]), dim=1).t()
        # edge: 2*D x E

        edge_e = self.leakyrelu(self.a.mm(edge_h).squeeze())

        e = torch.sparse_coo_tensor(edge, edge_e, (N,N))

        attention = F.softmax(e)
        attention = F.dropout(attention, self.dropout,training=self.training)

        h_prime = torch.matmul(attention,h)
        assert not torch.isnan(h_prime).any()
        
        return F.elu(h_prime)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [11]:
N, _F = 4,5
input = torch.randn((N,_F))
print(input)
e1 = torch.cat((torch.tensor(torch.arange(N-1),dtype=torch.long).view(1,N-1), torch.full((1,N-1),N-1,dtype=torch.long)))
e2 = torch.cat((torch.tensor(torch.arange(N),dtype=torch.long).view(1,N), torch.tensor(torch.arange(N),dtype=torch.long).view(1,N)))
edge = torch.cat((e1,e2),dim=1)
print(edge)

tensor([[ 1.2189,  0.2228,  1.0473, -0.0673, -1.1837],
        [ 1.8080, -1.4209,  0.6016, -0.5901,  0.5849],
        [ 1.5013, -1.8966,  0.1204, -0.2390,  0.4941],
        [-0.8741,  0.6490,  0.3912,  0.8649,  0.8556]])


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


tensor([[0, 1, 2, 0, 1, 2, 3],
        [3, 3, 3, 0, 1, 2, 3]])


In [12]:
sp_att = SpGraphAttentionLayer(_F,_F*2,0.5,0.01)

In [13]:
s = sp_att(input,edge)
print(s[:-1])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: Didn't find kernel to dispatch to for operator 'aten::_softmax'. Tried to look up kernel for dispatch key 'SparseCPUTensorId'. Registered dispatch keys are: [CUDATensorId, CPUTensorId, VariableTensorId, MkldnnCPUTensorId]